# Notebook Data Mining


## But  
Il s'agit, à partir des clusters déterminés par Kmean, d'identifier les bières similaires entre elles. 

Le clustering en lui-même n'apporte en effet pas directement ces informations, puisque une bière se retrouve souvent dans plusieurs clusters qui représentent des types de consommateurs.

## Démarche
Nous allons travailler sur les dataframes du clustering afin de lancer un algorithme de data mining, l'algorithme Apriori. Cet algorithme prend en entrée une liste de listes, où chacune des listes internes comprend des bières qui sont communes, c'est-à-dire des bières qu'on pourrait recommander entre elles à un consommateur sur la base de leurs similitudes déterminées à partir des avis subjectifs des consommateurs (base reviews). 

In [1]:
# Importation des modules nécessaires
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [3]:
df_subj_abv = pd.read_csv(r'C:\Users\sim13\OneDrive\Documents\Projet_python_2A\data_all_clusters.csv')
df_subj_abv

df_subj_abv = df_subj_abv.drop(['Unnamed: 0', 'index', 'look', 'smell', 'taste', 'feel'], axis = 'columns')
df_subj_abv = df_subj_abv.sample(10**3)

On va créer un data frame par cluster, afin de pouvoir tester les lignes de codes plus rapidement et de manière séparée.

In [4]:
cluster_all = df_subj_abv[['id', 'beer_name', 'beer_style', 'abv', 'brewery_id', 'brewery_name', 'cluster_all']]
cluster_subj = df_subj_abv[['id', 'beer_name', 'beer_style', 'abv', 'brewery_id', 'brewery_name', 'cluster_subj']]
cluster_nolook = df_subj_abv[['id', 'beer_name', 'beer_style', 'abv', 'brewery_id', 'brewery_name', 'cluster_nolook']]
cluster_feel_taste = df_subj_abv[['id', 'beer_name', 'beer_style', 'abv', 'brewery_id', 'brewery_name', 'cluster_feel/taste']]
cluster_feel_taste = cluster_feel_taste.rename(columns = {'cluster_feel/taste' : 'cluster_feel_taste'}) # Pour éliminer le problème avec le caractère '/'
cluster_taste_smell = df_subj_abv[['id', 'beer_name', 'beer_style', 'abv', 'brewery_id', 'brewery_name', 'cluster_taste/smell']]
cluster_taste_smell = cluster_taste_smell.rename(columns = {'cluster_taste/smell' : 'cluster_taste_smell'})

On crée une liste de ces dataframes, de leurs noms et un dictionnaire répertoriant le nombre de clusters de chacun des dataframes. Ils seront utiles pour l'automatisation de la mise en forme des données pour Apriori.

In [5]:
clusters_list = [cluster_all, cluster_subj, cluster_nolook, cluster_feel_taste, cluster_taste_smell]

clusters_name = ['cluster_all', 'cluster_subj', 'cluster_nolook', 'cluster_feel_taste', 'cluster_taste_smell']

clusters_nb = {}
for name, clus in zip(clusters_name, clusters_list):
    clusters_nb[name] = clus[name].nunique()
clusters_nb

{'cluster_all': 55,
 'cluster_subj': 53,
 'cluster_nolook': 26,
 'cluster_feel_taste': 15,
 'cluster_taste_smell': 14}

In [6]:
#!pip install apyori
from apyori import apriori

In [27]:
# Automatisation

list_apriori = []

for clus, clus_name in zip(clusters_list, clusters_name):
    all_cluster = []
    c = clusters_nb[clus_name]
    for i in range(0, c):
        dico_ex = {}
        listex = clus[clus[clus_name] == i].groupby('id')['id'].count()
        listex = np.array(listex)
        liste_id_beer = clus[clus[clus_name] == i].id.sort_values().unique()
        n = len(liste_id_beer)
        for j in range(0, n):
            dico_ex[liste_id_beer[j]] = listex[j]

        try:
            s = max(dico_ex[i] for i in dico_ex)
        except ValueError:
            break
        
        for i in range(s): 
            nb_nan, nb_id = 0, 0
            ticket = []
            for id_beer in dico_ex:
                if dico_ex[id_beer] >= 1:
                    ticket.append(str(id_beer))
                    dico_ex[id_beer] -= 1
                    nb_id += 1
                else:
                    #ticket.append('nan')
                    nb_nan += 1
                
            if nb_nan <= nb_id:
                all_cluster.append(ticket) # On rempli seulement s'il y a plus de nan que de bières pour que ce soit représentatif
            else:
                break
                
    list_apriori += all_cluster

## Application de l'algorithme Apriori et mise en forme des associations des bières

In [22]:
# Automatisation

list_apriori = all_cluster_all + all_cluster_subj + all_cluster_nolook + all_cluster_feel_taste + all_cluster_taste_smell


    
association_rules = apriori(clus, min_support=0.01, min_confidence=0.5, min_lift=3, min_length=2)

association_results = {}
frozen_first = next(association_rules)[0]
associated_first = []
for k in frozen_first:
    associated_first.append(k)
association_results[associated_first[0]] = [associated_first[1]]

associated = [0, 0]
while not(associated[-1] == associated_first[-1] and associated[0] == associated_first[0]): # Trouver la taille du générateur
    try:
        frozen = next(association_rules)[0]
    except StopIteration:
        break
    associated = []
    for k in frozen:
        associated.append(k)
        
    if associated[0] in association_results.keys():
        association_results[associated[0]].append(associated[1])
        
    elif associated[1] in association_results.keys():
        association_results[associated[1]].append(associated[0])
        
    else:
        association_results[associated[0]] = [associated[1]]
            
df_association_results = pd.DataFrame.from_dict(association_results, orient = 'index')
df_association_result = df_association_results.T
df_association_result.to_csv(r'C:\Users\sim13\OneDrive\Documents\Projet_python_2A\association_rules_dimil.csv')

In [74]:
# Essai avec all_cluster_subj
association_rules = apriori(all_cluster_subj, min_support=0.01, min_confidence=0.5, min_lift=3, min_length=2)

association_results = {}
frozen_first = next(association_rules)[0]
associated_first = []
for k in frozen_first:
    associated_first.append(k)
association_results[associated_first[0]] = [associated_first[1]]

associated = [0, 0]
while not(associated[-1] == associated_first[-1] and associated[0] == associated_first[0]): # Trouver la taille du générateur
    frozen = next(association_rules)[0]
    associated = []
    for k in frozen:
        associated.append(k)
        
    if associated[0] in association_results.keys():
        association_results[associated[0]].append(associated[1])
        
    elif associated[1] in association_results.keys():
        association_results[associated[1]].append(associated[0])
        
    else:
        association_results[associated[0]] = [associated[1]]

In [77]:
df_association_results = pd.DataFrame.from_dict(association_results, orient = 'index')
df_association_result = df_association_results.T
df_association_result

,100290,114236,100443,1005,1010,10119,1013,101601,102895,103016,...,782,82482,95746,689,76866,86621,92473,9794,77556,77897
0,104,100443,119759,114168,1576,128641,169625,104722,103549,110242,...,58914,58914,58914,693,693,71501,71501,71501,740,95520
1,109105,119759,120155,1144,16046,135796,248001,117212,106188,110808,...,64228,None,64228,76866,None,92473,None,86621,None,None
2,125646,120155,125641,124423,2339,22809,331329,12599,107328,1118,...,82482,None,782,None,None,None,None,92473,None,None
3,146164,125641,125646,148052,3566,25880,65,152902,11070,119745,...,None,None,82482,None,None,None,None,None,None,None
4,1577,125646,126669,189180,46290,3877,68690,167011,114884,131,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
102,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
103,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
104,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


La fonction apriori renvoie un objet de type générateur. Ce type d'objet sert à créer des itérateur et on accède à ces éléments avec l'instruction next. On trouva à l'intérieur de ces itération deux informations intéressantes : 
- un frozenset, type non-mutable, qui contient les deux bières associées, repérées par leur id
- un float qui contient un coefficient représentatif de l'associativité des 2 bières du fronzenset.

Concaténer ces informations dans une liste ou tout autre type plus lisible serait une perte de temps (c'est d'ailleurs très long). On va donc se contenter de conserver chacun des générateurs en mémoire et accéder à ce qui nous intéresse en temps voulu.

On peut essayer de les mettre dans un dictionnaire et mettre sous forme DF puis groupby et on aura toutes les bières associées à une bière donnée.

Trpuer la longueur du générateur !!

In [30]:
df_simil = pd.read_csv(r'C:\Users\sim13\OneDrive\Documents\Projet_python_2A\association_rules.csv')
df_simil.describe()

,Unnamed: 0,103016,100290,104867,10672,143023,166349,18850,213487,251573,...,9182,96579,8772,961,71973,86149,96814,88116,80379,9755
count,83.000000,53.000000,46.000000,1.0,21.000000,11.000000,17.000000,16.000000,8.000000,2.000000,...,5.000000,6.000000,1.0,2.000000,3.000000,2.000000,1.0,1.0,2.00000,1.0
mean,41.000000,101042.207547,100463.456522,100421.0,132318.333333,140316.727273,128467.764706,135318.875000,154363.875000,102644.000000,...,70867.400000,60586.500000,59469.0,34120.500000,83973.666667,82886.000000,68958.0,73393.0,42295.00000,74835.0
std,24.103942,86526.991607,92474.036356,NaN,105807.775523,106183.569176,102137.037224,101372.097486,110766.269826,3143.796749,...,9984.626923,26719.574261,NaN,35848.192486,14054.829787,19697.166497,NaN,NaN,46018.50932,NaN
min,0.000000,260.000000,260.000000,100421.0,645.000000,10672.000000,7520.000000,7520.000000,41815.000000,100421.000000,...,58299.000000,9182.000000,59469.0,8772.000000,68958.000000,68958.000000,68958.0,73393.0,9755.00000,74835.0
25%,20.500000,35738.000000,30195.750000,100421.0,46230.000000,69131.000000,63249.000000,72072.000000,72072.000000,101532.500000,...,66436.000000,60333.250000,59469.0,21446.250000,77553.500000,75922.000000,68958.0,73393.0,26025.00000,74835.0
50%,41.000000,87016.000000,61679.500000,100421.0,100421.000000,100421.000000,95028.000000,97724.500000,102644.000000,102644.000000,...,69522.000000,67979.000000,59469.0,34120.500000,86149.000000,82886.000000,68958.0,73393.0,42295.00000,74835.0
75%,61.500000,150877.000000,170539.750000,100421.0,220224.000000,232530.000000,213487.000000,223008.500000,255565.000000,103755.500000,...,74986.000000,73620.000000,59469.0,46794.750000,91481.500000,89850.000000,68958.0,73393.0,58565.00000,74835.0
max,82.000000,293916.000000,293916.000000,100421.0,330713.000000,330432.000000,330432.000000,330432.000000,330432.000000,104867.000000,...,85094.000000,85094.000000,59469.0,59469.000000,96814.000000,96814.000000,68958.0,73393.0,74835.00000,74835.0
